<a href="https://colab.research.google.com/github/ylmgrbzz/Deep-Learning-Projects-With-Keras-TensorFlow-PyTorch/blob/main/Titanic_Dataset_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import functools
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

In [3]:
train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv",  TEST_DATA_URL)

13049/13049 [==============================] - 0s 0us/step


In [4]:
train_df = pd.read_csv(train_file_path, header='infer')
test_df = pd.read_csv(test_file_path, header='infer')

In [5]:
train_df.head()


,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [7]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]
train_ds = tf.data.experimental.make_csv_dataset(
        train_file_path,
        batch_size = 3,
        label_name=LABEL_COLUMN,
        na_value="?",
        num_epochs= 1,
        ignore_errors=True)
test_ds = tf.data.experimental.make_csv_dataset(
        test_file_path,
        batch_size=3,
        label_name=LABEL_COLUMN,
        na_value="?",
        num_epochs=1,
        ignore_errors=True)

In [8]:
for batch, label in train_ds.take(1):
    print(label)
    for key, value in batch.items():
        print(f"{key}: {value.numpy()}")

tf.Tensor([0 1 0], shape=(3,), dtype=int32)
sex: [b'female' b'female' b'male']
age: [31. 18. 42.]
n_siblings_spouses: [0 2 0]
parch: [0 2 0]
fare: [  7.8542 262.375    7.55  ]
class: [b'Third' b'First' b'Third']
deck: [b'unknown' b'B' b'unknown']
embark_town: [b'Southampton' b'Cherbourg' b'Southampton']
alone: [b'y' b'n' b'y']


In [9]:
feature_columns = []
for header in ['age', 'n_siblings_spouses', 'parch', 'fare']:
    feature_columns.append(feature_column.numeric_column(header))

In [10]:
titanic_df = pd.read_csv(train_file_path, header='infer')
titanic_df.describe()

,survived,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000,627.000000
mean,0.387560,29.631308,0.545455,0.379585,34.385399
std,0.487582,12.511818,1.151090,0.792999,54.597730
min,0.000000,0.750000,0.000000,0.000000,0.000000
25%,0.000000,23.000000,0.000000,0.000000,7.895800
50%,0.000000,28.000000,0.000000,0.000000,15.045800
75%,1.000000,35.000000,1.000000,0.000000,31.387500
max,1.000000,80.000000,8.000000,5.000000,512.329200


In [11]:
age = feature_column.numeric_column('age')
age_buckets = feature_column.bucketized_column(age, boundaries=[23, 28, 35])

In [12]:
h = {}
for col in titanic_df:
    if col in ['sex', 'class', 'deck', 'embark_town', 'alone']:
        print(col, ':', titanic_df[col].unique())
        h[col] = titanic_df[col].unique()

sex : ['male' 'female']
class : ['Third' 'First' 'Second']
deck : ['unknown' 'C' 'G' 'A' 'B' 'D' 'F' 'E']
embark_town : ['Southampton' 'Cherbourg' 'Queenstown' 'unknown']
alone : ['n' 'y']


In [13]:
sex_type = feature_column.categorical_column_with_vocabulary_list(
      'Type', h.get('sex').tolist())
sex_type_one_hot = feature_column.indicator_column(sex_type)

class_type = feature_column.categorical_column_with_vocabulary_list(
      'Type', h.get('class').tolist())
class_type_one_hot = feature_column.indicator_column(class_type)

deck_type = feature_column.categorical_column_with_vocabulary_list(
      'Type', h.get('deck').tolist())
deck_type_one_hot = feature_column.indicator_column(deck_type)

embark_town_type = feature_column.categorical_column_with_vocabulary_list(
      'Type', h.get('embark_town').tolist())
embark_town_type_one_hot = feature_column.indicator_column(embark_town_type)

alone_type = feature_column.categorical_column_with_vocabulary_list(
      'Type', h.get('alone').tolist())
alone_one_hot = feature_column.indicator_column(alone_type)

In [14]:
deck = feature_column.categorical_column_with_vocabulary_list(
      'deck', titanic_df.deck.unique())
deck_embedding = feature_column.embedding_column(deck, dimension=3)

In [15]:
class_hashed = feature_column.categorical_column_with_hash_bucket(
      'class', hash_bucket_size=4)

In [16]:
cross_type_feature = feature_column.crossed_column(['sex', 'class'], hash_bucket_size=5)


In [17]:
feature_columns = []


In [18]:
for header in ['age', 'n_siblings_spouses', 'parch', 'fare']:
    feature_columns.append(feature_column.numeric_column(header))
age = feature_column.numeric_column('age')
age_buckets = feature_column.bucketized_column(age, boundaries=[23, 28, 35])
feature_columns.append(age_buckets)

indicator_column_names = ['sex', 'class', 'deck', 'embark_town', 'alone']
for col_name in indicator_column_names:
    categorical_column = feature_column.categorical_column_with_vocabulary_list(
        col_name, titanic_df[col_name].unique())
    indicator_column = feature_column.indicator_column(categorical_column)
    feature_columns.append(indicator_column)
    
deck = feature_column.categorical_column_with_vocabulary_list(
      'deck', titanic_df.deck.unique())
deck_embedding = feature_column.embedding_column(deck, dimension=3)
feature_columns.append(deck_embedding)

feature_columns.append(feature_column.indicator_column(cross_type_feature))

In [19]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)  


In [20]:
val_df, test_df = train_test_split(test_df, test_size=0.4)


In [21]:
labels = train_df.pop("survived")


In [23]:
def pandas_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('survived')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [24]:
batch_size=32
val_ds = pandas_to_dataset(val_df, shuffle=False, batch_size=batch_size)
test_ds = pandas_to_dataset(test_df, shuffle=False, batch_size=batch_size)

In [27]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)])


In [28]:
model.compile(optimizer='adam', 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [29]:
model.compile(optimizer='adam', 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [30]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10


    206/Unknown - 4s 6ms/step - loss: 1.1228 - accuracy: 0.6553

209/209 [==============================] - 5s 8ms/step - loss: 1.1188 - accuracy: 0.6523 - val_loss: 0.6067 - val_accuracy: 0.6646
Epoch 2/10
209/209 [==============================] - 1s 7ms/step - loss: 0.8065 - accuracy: 0.6762 - val_loss: 0.6056 - val_accuracy: 0.7342
Epoch 3/10
209/209 [==============================] - 1s 6ms/step - loss: 0.6143 - accuracy: 0.7018 - val_loss: 0.5114 - val_accuracy: 0.7342
Epoch 4/10
209/209 [==============================] - 1s 6ms/step - loss: 0.7030 - accuracy: 0.7097 - val_loss: 0.5237 - val_accuracy: 0.7089
Epoch 5/10
209/209 [==============================] - 1s 6ms/step - loss: 0.5467 - accuracy: 0.7544 - val_loss: 0.4735 - val_accuracy: 0.7405
Epoch 6/10
209/209 [==============================] - 1s 6ms/step - loss: 0.4909 - accuracy: 0.7943 - val_loss: 0.6021 - val_accuracy: 0.6899
Epoch 7/10
209/209 [==============================] - 1s 6ms/step - loss: 0.5005 - accuracy: 0.7799 - val_loss: 0.4618 - val_accuracy: 0.7405
Epoch 8/10
209/20